# ДЛЯ БОЛЬШИХ ОБЪЕМОВ ДАННЫХ СЧИТАЕМ МЕТРИКИ ДЛЯ ОТРЫВКОВ ТЕКСТОВ А ПОТОМ УСРЕДНЯЕМ МЕТРКИ МЕЖДУ СОХРАНЕННЫМИ ФАЙЛАМИ

In [ ]:
https://medium.com/@nicharuch/collocations-identifying-phrases-that-act-like-individual-words-in-nlp-f58a93a2f84a
    
    https://github.com/nicharuc/Collocations/blob/master/Collocations.ipynb
        
        
https://www.nltk.org/_modules/nltk/tag.html
    
https://www.nltk.org/_modules/nltk/collocations.html
    
https://www.nltk.org/api/nltk.metrics.html

In [1]:
from calculate_colocations import get_pos_filtered_colloc_from_corpus_list, get_colloc_from_corpus_list
import pandas as pd
from tqdm import tqdm

In [2]:
texts_lenta = pd.read_csv("3000.csv")

texts_lenta.iloc[0]['texts_3000']

'Указом президента России Бориса Ельцина внесены изменения в  существующую структуру Федеральной службы безопасности РФ, утвержденную в июле прошлого года. Как говорится в поступившем сегодня сообщении Центра общественных связей ФСБ, в соответствии с Основами (Концепцией) государственной политики Российской Федерации по военномустроительству на период до 2005 года, на базе Департамента по борьбе с терроризмом и Управления конституционной безопасности ФСБ создан Департамент по защите конституционного строя и борьбе с терроризмом. В составе департамента организуются три управления с четко определенной компетенцией. В ФСБ отмечают, что "в современных условиях для российскойгосударственности имеют приоритетное значение вопросы защитыконституционного строя, сохранения целостности страны, борьбыс терроризмом и всеми формами экстремизма, а также разведывательно-подрывной деятельностью спецслужб и организаций иностранных государств". Как подчеркивается в сообщении, "органам безопасности в реше

# FREQ POS FILTER

In [3]:

bigramFreqTable, trigramFreqTable, quadgram_freq, filtered_bi, filtered_tri, bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable =get_pos_filtered_colloc_from_corpus_list(list(texts_lenta['texts_3000']),"rus")


100%|████████████████████████████████████████████████████████████████████████████████| 898/898 [04:26<00:00,  3.37it/s]


In [5]:
def cean_pos_tags(df, ngramm_name):
    clean_words = []
    for i in tqdm(range(len(df))):
        posed_ngramm = df.iloc[i][ngramm_name]
        clean_ngramm = []
        for w in posed_ngramm:
            word = w.split("_")[0]
            clean_ngramm.append(word)
        clean_ngramm = tuple(clean_ngramm)
        clean_words.append(clean_ngramm)
    
    data = { "ngramm":clean_words,"freq":df['freq']}
    clean_df = pd.DataFrame(data) 
    return clean_df
bigramFreqTable_clean = cean_pos_tags(bigramFreqTable,'bigram')


100%|██████████████████████████████████████████████████████████████████████████| 91240/91240 [00:24<00:00, 3746.55it/s]


In [6]:
trigramFreqTable_clean = cean_pos_tags(trigramFreqTable, "trigram")

100%|████████████████████████████████████████████████████████████████████████| 133933/133933 [00:38<00:00, 3512.21it/s]


In [8]:
quadgram_freq_clean = cean_pos_tags(quadgram_freq, "quadgramF")

100%|████████████████████████████████████████████████████████████████████████| 143433/143433 [00:41<00:00, 3449.58it/s]


In [15]:
def get_united_numeric_colloc_base (numeric_calc_collocations_list):#все кроме фильтрованных по частям речи
    
    overall_colloc_json = {'2':{},'3':{},'4':{}}
    for colloc in numeric_calc_collocations_list:
        colloc_len = len(colloc)
        for key in list(colloc.keys()):
            if 'gram' in key:
                ngramm_name = key
            else:
                freq_name = key
        n_of_words = str(len(colloc.iloc[0][ngramm_name]))
        print(n_of_words)
        for index in tqdm(range(int(colloc_len*0.5))):
            collocation_element = colloc.iloc[index][ngramm_name]
            str_el = ''
            for el in collocation_element:
                str_el += el + ' '
            str_el = str_el.strip()
            if str_el in overall_colloc_json[n_of_words]:
                overall_colloc_json[n_of_words][str_el] *= colloc.iloc[index][freq_name]
            else:
                overall_colloc_json[n_of_words][str_el] = colloc.iloc[index][freq_name]
            #print(str_el)
        #print("========")
    return overall_colloc_json
ngramms_list = [bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean, bigramPMITable, trigramPMITable, quadragramPMITable]
colloc_dict = get_united_numeric_colloc_base(ngramms_list)

2


100%|██████████████████████████████████████████████████████████████████████████| 45620/45620 [00:24<00:00, 1887.54it/s]


3


100%|██████████████████████████████████████████████████████████████████████████| 66966/66966 [00:33<00:00, 1977.37it/s]


4


...

100%|██████████████████████████████████████████████████████████████████████████████| 581/581 [00:00<00:00, 1655.20it/s]


3


100%|██████████████████████████████████████████████████████████████████████████████| 302/302 [00:00<00:00, 1745.73it/s]


4


100%|██████████████████████████████████████████████████████████████████████████████| 215/215 [00:00<00:00, 1660.31it/s]


In [22]:
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x_pmi = sorted(colloc_dict['2'].items(), key=lambda kv: kv[1], reverse = True)
sorted_x_pmi

[('то что', 36015.02077888884),
 ('владимир путин', 12741.956141008714),
 ('один из', 8868.579180891744),
 ('мочь быть', 6862.869928830487),
 ('не быть', 3741),
 ('то как', 2761.185645801102),
 ('время новость', 2466.6964373584024),
 ('и в', 2414),
 ('как сообщать', 2300.95103778089),
 ('отмечать что', 2087.5346320619806),
 ('пока не', 1973.3051858641375),
 ('должен быть', 1945.3199879491715),
 ('быть и', 1872),
 ('риа новость', 1644.9924678104478),
 ('министерство оборона', 1435.0687724781678),
 ('закон о', 1413.0987662429243),
 ('вооружать сила', 1358.551169189221),
 ('по слово', 1229.6163010916455),
 ('и не', 1210),
 ('о это', 1177.6563579652332),
 ('при это', 1101.1265529917937),
 ('а также', 1011.4236227397097),
 ('кроме то', 936.4248787183583),
 ('заявлять что', 833.2330993440846),
 ('сообщать агентство', 784.8468890943133),
 ('вооруженный сила', 747.7064732082199),
 ('что это', 736),
 ('что в', 732),
 ('что он', 704),
 ('он слово', 700.2131993633352),
 ('тот число', 628.81015263

In [ ]:
bigramFreqTable.info()

In [ ]:
trigramPMITable.iloc[0]

In [ ]:
for key in list(trigramPMITable.keys()):
    if 'gram' in key:
        print(key)

In [ ]:
trigramPMITable.head(50)

In [ ]:
clean_words = []
for i in tqdm(range(len(filtered_tri))):
    posed_ngramm = filtered_tri.iloc[i]['trigram']
    clean_ngramm = []
    for w in posed_ngramm:
        word = w.split("_")[0]
        clean_ngramm.append(word)
    clean_ngramm = tuple(clean_ngramm)
    clean_words.append(clean_ngramm)

In [ ]:
trigramChiTable.to_csv("trigramChiTable.csv")

In [ ]:
#data = { "trigram":clean_words,"freq":filtered_tri['freq']}
#filtered_tri_clean = pd.DataFrame(data) 
filtered_tri_clean.head()

In [ ]:
#data = { "filtered_bi":clean_words,"freq":filtered_bi['freq']}
#filtered_bi_clean = pd.DataFrame(data) 
filtered_bi_clean.head()

In [ ]:
#data = { "trigram":clean_words,"freq":quadgram_freq['freq']}
#quadgram_freq_clean = pd.DataFrame(data) 
quadgram_freq_clean.head()

In [ ]:
#data = { "trigram":clean_words,"freq":trigramFreqTable['freq']}
#trigramFreqTable_clean = pd.DataFrame(data) 
trigramFreqTable_clean.head()

In [ ]:
#data = { "bigram":clean_words,"freq":bigramFreqTable['freq']}
#bigramFreqTable_clean = pd.DataFrame(data) 
bigramFreqTable_clean.head()

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
model_en = Model('english-partut-ud-2.0-170801.udpipe')
texts_en = pd.read_csv("eng_texts.csv")
texts_list = list(texts_en['page_text'])
bigramFreqTable, trigramFreqTable, quadgram_freq, filtered_bi, filtered_tri, bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable =get_pos_filtered_colloc_from_corpus_list(texts_list, stopwords, model_en)


In [ ]:
quadragramPMITable

# FREQ PMI ETC

In [ ]:
texts_lenta = pd.read_csv("3000.csv")
corpus_list = list(texts_lenta['texts_3000'])
bigramFreqTable, trigramFreqTable,quadgram_freq,  bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable = get_colloc_from_corpus_list(corpus_list, "rus")


In [ ]:
bigramChiTable

In [ ]:
texts_en = pd.read_csv("eng_texts.csv")
texts_list_en = list(texts_en['page_text'])
bigramFreqTable, trigramFreqTable,quadgram_freq,  bigramPMITable, trigramPMITable, quadragramPMITable , bigramChiTable, trigramChiTable= get_colloc_from_corpus_list(texts_list_en, "en")



In [ ]:
quadragramPMITable